# TDDA: Test-Driven Data Analysis

[TDDA](https://github.com/tdda/tdda) verwendet Dateieingaben (wie NumPy-Arrays oder Pandas DataFrames) und eine Reihe von Einschränkungen (engl.: _constraints_), die als JSON-Datei gespeichert werden.

* [Reference Test](https://tdda.readthedocs.io/en/latest/referencetest.html) unterstützt die Erstellung von Referenztests, die entweder auf `unittest` oder `pytest` basieren.
* [Constraints](https://tdda.readthedocs.io/en/v1.0.30/constraints.html) wird verwendet, um Constraints aus einem (Pandas)-DataFrame zu ermitteln, sie als JSON auszuschreiben und zu überprüfen, ob Datensätze die Constraints in der Constraints-Datei erfüllen. Es unterstützt auch Tabellen in einer Vielzahl von relationalen Datenbanken.
* [Rexpy](https://tdda.readthedocs.io/en/v1.0.30/rexpy.html) ist ein Werkzeug zur automatischen Ableitung von regulären Ausdrücken aus einer Spalte in einem Pandas DataFrame oder aus einer (Python)-Liste von Beispielen.

## 1. Importe

In [1]:
import pandas as pd
import numpy as np

from tdda.constraints import discover_df, verify_df, detect_df

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/kjam/data-cleaning-101/master/data/iot_example.csv")

## 2. Daten überprüfen

Mit [pandas.DataFrame.sample](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) lassen wir uns die ersten zehn Datensätze anzeigen:

In [3]:
df.sample(10)

,timestamp,username,temperature,heartrate,build,latest,note
128949,2017-02-22T00:25:13,agonzales,17,82,79f4d638-9519-b55d-d0fd-0414d872330f,0,sleep
4304,2017-01-03T05:25:20,aguilarryan,11,80,2904318b-9225-f8ab-97b6-64864f1506b3,0,test
141120,2017-02-26T21:14:24,carterdavid,18,66,1dca8f4e-41ea-c158-cd1a-d0b2e4447558,0,sleep
40376,2017-01-17T15:11:42,ymoore,29,60,6885fd47-9931-bf16-b176-9eb70f06915b,0,interval
140046,2017-02-26T10:56:27,mgreen,11,87,776b59c4-901a-557f-499a-04c14824622e,0,NaN
23149,2017-01-10T17:41:07,caleb85,18,83,4c7f9811-fd1d-08d9-e3a5-fd5061b593b1,1,user
31767,2017-01-14T04:33:02,udelgado,14,71,76223faa-be13-8898-84ef-53adad97a47b,1,interval
52543,2017-01-22T11:53:00,ivargas,22,64,18390eec-ed4e-ba5f-1124-fde483453a24,0,NaN
56781,2017-01-24T04:30:59,wbrown,9,67,e860bce6-9999-eb9d-fabe-37f870e7fb52,1,interval
132476,2017-02-23T10:27:05,ygutierrez,13,61,626d4d0b-5d9a-4886-3e6a-179fa8c313ff,0,update


Und mit [pandas.DataFrame.dtypes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html) lassen wir uns die Datentypen für die einzelnen Spalten anzeigen:

In [4]:
df.dtypes

timestamp      object
username       object
temperature     int64
heartrate       int64
build          object
latest          int64
note           object
dtype: object

## 3. Erstellen eines _constraints_-Objekt

Mit `discover_constraints` kann ein Vonstraints-Objekt erzeugt werden.

In [5]:
constraints = discover_df(df)

In [6]:
constraints

In [7]:
constraints.fields

Fields([('timestamp', <tdda.constraints.base.FieldConstraints at 0x147df6250>),
        ('username', <tdda.constraints.base.FieldConstraints at 0x147df6790>),
        ('temperature',
         <tdda.constraints.base.FieldConstraints at 0x147df6bd0>),
        ('heartrate', <tdda.constraints.base.FieldConstraints at 0x147df7250>),
        ('build', <tdda.constraints.base.FieldConstraints at 0x147df76d0>),
        ('latest', <tdda.constraints.base.FieldConstraints at 0x147df7e10>),
        ('note', <tdda.constraints.base.FieldConstraints at 0x147dfc2d0>)])

## 4. Schreiben der _Constraints_ in eine Datei

In [8]:
with open("iot_example.json", "w") as f:
    f.write(constraints.to_json())

Wenn wir uns die Datei genauer betrachten können wir erkennen, dass z.B. für die `timestamp`-Spalte eine Zeichenkette mit 19 Zeichen erwartet wird und `temperature` Integer mit Werten von 5–29 erwartet.

In [9]:
cat iot_example.json

{
    "creation_metadata": {
        "local_time": "2025-12-16 12:12:52",
        "utc_time": "2025-12-16 11:11:52",
        "creator": "TDDA 2.0.09",
        "host": "fay.local",
        "user": "veit",
        "n_records": 146397,
        "n_selected": 146397
    },
    "fields": {
        "timestamp": {
            "type": "string",
            "min_length": 19,
            "max_length": 19,
            "max_nulls": 0,
            "no_duplicates": true
        },
        "username": {
            "type": "string",
            "min_length": 3,
            "max_length": 21,
            "max_nulls": 0
        },
        "temperature": {
            "type": "int",
            "min": 5,
            "max": 29,
            "sign": "positive",
            "max_nulls": 0
        },
        "heartrate": {
            "type": "int",
            "min": 60,
            "max": 89,
            "sign": "positive",
            "max_nulls": 0
        },
        "build": {
            "type": "string"

## 5. Überprüfen von Dataframes

Hierfür lesen wir zunächst eine neue csv-Datei mit Pandas ein und lassen uns dann zehn Datensätze exemplarisch ausgeben:

In [10]:
new_df = pd.read_csv("https://raw.githubusercontent.com/kjam/data-cleaning-101/master/data/iot_example_with_nulls.csv")

new_df.sample(10)

,timestamp,username,temperature,heartrate,build,latest,note
22234,2017-01-10T08:57:07,jason63,10.0,70,2409db76-ce0b-5696-28d8-3864b0cfd3ea,NaN,NaN
19100,2017-01-09T02:59:27,onealjennifer,15.0,80,488f7adb-3fe1-442d-7730-30ce925b8cd6,0.0,wake
95486,2017-02-08T16:16:49,andrew33,5.0,88,NaN,0.0,test
92111,2017-02-07T07:57:59,stefanie36,22.0,62,faf9dbc3-34ea-57a1-613c-d3a2baff6bd8,0.0,sleep
141431,2017-02-27T00:12:35,reyeserika,13.0,70,e6b479b2-f2fe-87a8-f017-049d0ac83140,0.0,NaN
67982,2017-01-28T16:14:51,bailey22,10.0,67,275255dd-318e-b2de-a0fd-fe7778be29cd,NaN,wake
78986,2017-02-02T02:11:55,bwilson,20.0,67,NaN,NaN,sleep
88370,2017-02-05T20:02:02,jennifer80,17.0,60,NaN,1.0,NaN
15968,2017-01-07T21:15:23,sabrina07,26.0,89,d502481a-e6f7-8cb6-9243-ce367076689e,NaN,test
130568,2017-02-22T15:58:05,warrennicholas,28.0,60,95838fd9-5ee5-8a17-fec7-602cecb6c48b,NaN,NaN


Wir sehen mehrere Felder, die als `NaN` ausgegeben werden. Um dies nun systematisch zu analysieren, wenden wir [verify_df](https://tdda.readthedocs.io/en/v1.0.31/constraints.html#tdda.constraints.verify_df) auf unseren neuen DataFrame an. Dabei gibt `passes` gibt die Anzahl der bestandenen, `failures` die Anzahl der fehlgeschlagenen Constraints zurück.

In [11]:
v = verify_df(new_df, "iot_example.json")

In [12]:
v

In [13]:
v.passes

30

In [14]:
v.failures

3

Wir können uns auch anzeigen lassen, in welchen Spalten welche Constraints bestanden und fehlgeschlagen sind: 

In [15]:
print(str(v))

FIELDS:

timestamp: 0 failures  5 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✓  no_duplicates ✓

username: 0 failures  4 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✓

temperature: 1 failure  4 passes  type ✓  min ✓  max ✓  sign ✓  max_nulls ✗

heartrate: 0 failures  5 passes  type ✓  min ✓  max ✓  sign ✓  max_nulls ✓

build: 1 failure  4 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✗  no_duplicates ✓

latest: 1 failure  4 passes  type ✓  min ✓  max ✓  sign ✓  max_nulls ✗

note: 0 failures  4 passes  type ✓  min_length ✓  max_length ✓  allowed_values ✓

SUMMARY:

Constraints passing: 30
Constraints failing: 3


Alternativ können wir uns diese Ergebnisse auch tabellarisch anzeigen lassen:

In [16]:
v.to_frame()

,field,failures,passes,type,min,min_length,max,max_length,sign,max_nulls,no_duplicates,allowed_values
0,timestamp,0,5,True,NaN,True,NaN,True,NaN,True,True,NaN
1,username,0,4,True,NaN,True,NaN,True,NaN,True,NaN,NaN
2,temperature,1,4,True,True,NaN,True,NaN,True,False,NaN,NaN
3,heartrate,0,5,True,True,NaN,True,NaN,True,True,NaN,NaN
4,build,1,4,True,NaN,True,NaN,True,NaN,False,True,NaN
5,latest,1,4,True,True,NaN,True,NaN,True,False,NaN,NaN
6,note,0,4,True,NaN,True,NaN,True,NaN,NaN,NaN,True


## 6. Finden der fehlerhaften Zeilen

`tdda.constraints.pd.constraints.detect_df()` erkennt Datensätze des pandas DataFrame, die gegen eine der Einschränkungen in der bereitgestellten JSON-Datei verstoßen. Anschließend können wir über dem erstellten `PandasDetection`-Objekt die Funktion `detected()` aufrufen um uns die Zeilen ausgeben zu lassen, die fehlerhaft sind:

In [17]:
d = detect_df(new_df, "iot_example.json")

d.detected()

,n_failures
Index,
3,1
4,1
7,1
10,2
12,1
...,...
146385,1
146387,2
146391,2


Wir können uns alle fehlerhaften Datensätze anzeigen lassen, indem wir nur den Teil des Index von `new_df` verwenden, der auch in `d.detected()` vorkommt:

In [18]:
d_index = d.detected().index

In [19]:
new_df[new_df.index.isin(d_index)]

,timestamp,username,temperature,heartrate,build,latest,note
3,2017-01-01T12:02:09,eddierodriguez,28.0,76,NaN,0.0,update
4,2017-01-01T12:02:36,kenneth94,29.0,62,122f1c6a-403c-2221-6ed1-b5caa08f11e0,NaN,NaN
7,2017-01-01T12:04:35,scott28,16.0,76,7a60219f-6621-e548-180e-ca69624f9824,NaN,interval
10,2017-01-01T12:06:21,njohnson,NaN,63,e09b6001-125d-51cf-9c3f-9cb686c19d02,NaN,NaN
12,2017-01-01T12:07:41,jessica48,22.0,83,03e1a07b-3e14-412c-3a69-6b45bc79f81c,NaN,update
...,...,...,...,...,...,...,...
146385,2017-02-28T23:53:59,powelleric,20.0,86,152eda10-676a-069c-b664-19443f2c8081,NaN,test
146387,2017-02-28T23:54:50,jthompson,NaN,66,8da10303-fe49-e313-8fda-0d5e79ded054,NaN,update
146391,2017-02-28T23:57:21,aaronbecker,NaN,87,7e52f4a8-345c-5ee0-e515-b8c392213062,NaN,sleep
146393,2017-02-28T23:58:43,joelrusso,NaN,89,NaN,0.0,NaN


Alternativ können wir uns auch alle fehlerfreien Datensätze anzeigen lassen.

In [20]:
new_df[~new_df.index.isin(d_index)]

,timestamp,username,temperature,heartrate,build,latest,note
0,2017-01-01T12:00:23,michaelsmith,12.0,67,4e6a7805-8faa-2768-6ef6-eb3198b483ac,0.0,interval
1,2017-01-01T12:01:09,kharrison,6.0,78,7256b7b0-e502-f576-62ec-ed73533c9c84,0.0,wake
2,2017-01-01T12:01:34,smithadam,5.0,89,9226c94b-bb4b-a6c8-8e02-cb42b53e9c90,0.0,NaN
5,2017-01-01T12:03:04,bryanttodd,13.0,86,0897dbe5-9c5b-71ca-73a1-7586959ca198,0.0,interval
6,2017-01-01T12:03:51,andrea98,17.0,81,1c07ab9b-5f66-137d-a74f-921a41001f4e,1.0,NaN
...,...,...,...,...,...,...,...
146389,2017-02-28T23:56:05,kathy63,5.0,88,c2f76050-abd4-aee4-7bc0-3498325d0573,0.0,NaN
146390,2017-02-28T23:56:34,cookallison,16.0,84,f0b0c1f9-900b-276c-bca9-ac4d4ec4e88e,0.0,user
146392,2017-02-28T23:58:06,mcontreras,15.0,63,69e61a15-d2d0-47a7-1a27-e07b3eeeba10,0.0,NaN
146395,2017-02-28T23:59:48,grayjasmin,17.0,64,4911a589-3a15-4bbf-1de1-e5a69ab739da,1.0,update
